### Basic pydantic example and explanation

This is a working through and documentation of the basic example on the pydantic home page. If you want to see the original go [here](https://pydantic-docs.helpmanual.io/)

In [45]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, ValidationError

"""This is the pydantic model
INFO:
- Each attribute is given an explicit type
- Optional[?] = None will always be used when
the default value is None. When the default
value is None it implicity means that the
argument is optionaly. But when we use
Optional, we are being explicit
- Every attribute not optional is required
- The main thing that gets pydantic working,
is Class user inherits from pydantic class
BaseModel
- You can think of models as similar to types in
strictly typed languages, or as the requirements
of a single endpoint in an API
- we don't do stuff like, self.id = id. We just
need to supply User with id and it knows what to
do with it
- the default type of name will be infered to be
str from the default value
- we can be explicit like: name: str = 'John Doe'
if we want to
- even though name has a default value it isn't
forced to this value, you can supply a different
name

NOTE:
- pydantic is primarily a parsing library, not a
validation library. Validation is a means to an
end: building a model which conforms to the types
and constraints provided
"""
class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []


external_data = {
    'id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}

"""Creating a user instance
- **external_data is saying out of this dict
find the keys used by the function (kwargs).
This isn't a pydantic thing, it's a python
thing
- The values you provide will be type casted
to the type of the attribute. If this can't happen
then an error will be raised
"""
user = User(**external_data)

"""
- user is an instance of class User. As such
the attributes like id can be retrieved like
any other class (user.id)
- user.dict() == dict(user). We prefect
user.dict(), because you can provide some
useful arguments (aparently)
"""
print('id:', user.id)
print('signup_ts:', repr(user.signup_ts))
print('friends:', user.friends)
print('dict():', user.dict())

"""This is an example of how pydantic catches all your errors
- using the pydantic ValidationError
"""
try:
    User(signup_ts='broken', friends=[None, 2, 'not number'])
except ValidationError as e:
    print('\ne.json():', e.json())

id: 123
signup_ts: datetime.datetime(2019, 6, 1, 12, 22)
friends: [1, 2, 3]
dict(): {'id': 123, 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'friends': [1, 2, 3], 'name': 'John Doe'}

e.json(): [
  {
    "loc": [
      "id"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "signup_ts"
    ],
    "msg": "invalid datetime format",
    "type": "value_error.datetime"
  },
  {
    "loc": [
      "friends",
      0
    ],
    "msg": "none is not an allowed value",
    "type": "type_error.none.not_allowed"
  },
  {
    "loc": [
      "friends",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  }
]


### More pydantic

This code comes from [Usage > Models](https://pydantic-docs.helpmanual.io/usage/models/) in the pydantic docs

In [60]:
from pydantic import BaseModel

class Cat(BaseModel):
    id: int
    age: int
    male: Optional[bool] = None
    name = 'Jane Doe'

cat = Cat(id=0, age=3, male=True)

"""
- __fields_set__ is an attribute of
every pydantic model (inherited from BaseModel)
(I kind of guessed this)
- object.__fields_set__ returns a set of the
attribute names of the model

NOTE:
- name isn't in __fields_set__. This is because
the default name vale is being used. If a name
was provided, then name would appear in
__fields_set__
"""
print('__fields_set__:')
print(cat.__fields_set__)

"""
- __fields__ is an attribute of
every pydantic model (inherited from BaseModel)
(I kind of guessed this)
- object.__fields__ returns a dict that
explains all of the fields in the model
- the dict uses str keys and values of
pydantic.fields.ModelField
"""
print('\n__fields__:')
print(cat.__fields__)

"""
- the configuration class for the model
- I think this is only useful when
you get heavy into pydantic
"""
print('\n__config__:')
print(cat.__config__)

__fields_set__:
{'male', 'id', 'age'}

__fields__:
{'id': ModelField(name='id', type=int, required=True), 'age': ModelField(name='age', type=int, required=True), 'male': ModelField(name='male', type=Optional[bool], required=False, default=None), 'name': ModelField(name='name', type=str, required=False, default='Jane Doe')}

__config__:
<class '__main__.Config'>


### Nested pydantic

This code comes from [Usage > Models](https://pydantic-docs.helpmanual.io/usage/models/) in the pydantic docs

In [63]:
from typing import List
from pydantic import BaseModel


class Foo(BaseModel):
    count: int
    size: float = None


class Bar(BaseModel):
    apple = 'x'
    banana = 'y'

"""
- Spam uses Foo and Bar pydantic classes
for type casting. Which means these smaller
classes will be recursivly handled by pydantic
"""
class Spam(BaseModel):
    foo: Foo
    bars: List[Bar]


m = Spam(foo={'count': 4}, bars=[{'apple': 'x1'}, {'apple': 'x2'}])
print(m)
print(m.dict())

foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]
{'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}
